# Dot Size
This notebook is used to determine the appropriate size of the quantum dot in a dot superconductor dot system

In [1]:
try:
    import nqcpfem
except ModuleNotFoundError as err:
    import sys
        # put path to nqcpfem module here   
    src_path = '/mnt/c/Users/swao/nqcp/NQCP_FEM/'
    sys.path.append(src_path)
    import nqcpfem
import sympy
sympy.init_printing(use_latex='mathjax')
import IPython.display as disp
from matplotlib import pyplot as plt
import numpy as np
import os

import logging
LOG = logging.getLogger()
LOG.setLevel(logging.INFO)
from nqcpfem.parameter_search import DBMPParameterSearch, ParameterSearch
from nqcpfem.band_model import LuttingerKohnHamiltonian
from nqcpfem.solvers import PETScSolver
from nqcpfem.envelope_function import RectangleDomain
from nqcpfem.fenics import FEniCsModel
from nqcpfem.g_matrix import GMatrix
from nqcpfem.band_model import __MAGNETIC_FIELD_NAMES__ as _B_
_B_ = sympy.symbols(_B_,commutative=False) # cast to symbols
from nqcpfem.functions import SymbolicFunction





In [2]:
import os
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = 1

TypeError: str expected, not int

In [3]:
bm = LuttingerKohnHamiltonian(spatial_dim=3).material_spec('Ge')
bm.add_z_confinement(1,'box',25e-9)
#bm.add_zeeman_term()#B=[1,1,1])



solver = PETScSolver(k=10,which='SM',sigma=0)
domain = RectangleDomain(Lx=250e-9,Ly=150e-9,Lz=0) 
domain.resolution = [300,180]
fem_model = FEniCsModel(bm,domain,0,('CG',1))


x,y,z = bm.position_symbols
omega_x,omega_y = sympy.symbols(r'\omega_x,\omega_y')
m = sympy.symbols('m')

V = 0.5*m*( (omega_x*x)**2 + (omega_y*y)**2 ) 

bm.add_potential(V)

bm.material_spec('Ge')

E0 = 1/(nqcpfem.UNIT_CONVENTION['J to eV']*1000000) # unit_scale: 1muev
omega_y_val = 2e12
omega_x_val = omega_y_val


fem_model.band_model.parameter_dict[omega_y] = omega_y_val

fem_model.band_model.parameter_dict[omega_x] = omega_x_val


In [1]:
sol = solver.solve(fem_model)

NameError: name 'solver' is not defined

In [5]:
import dolfinx.fem as fem

In [7]:
from dolfinx.fem import petsc

<module 'dolfinx.fem.petsc' from '/home/osch/anaconda3/envs/ge_spin_fe/lib/python3.11/site-packages/dolfinx/fem/petsc.py'>